<h2>Aufbereiten der Datensätze</h2>

Es werden Die Span-based Amazon Datensätze von Gupta et. al (2019) (https://arxiv.org/pdf/1908.04364.pdf) verwendet (siehe https://github.com/amazonqa/amazonqa), die auf den bekannten Amazon QA Datensätzen von Julian Mcauley (2016) (https://cseweb.ucsd.edu//~jmcauley/pdfs/sigir15.pdf) basieren (siehe http://jmcauley.ucsd.edu/data/amazon/qa/).

Der TrainingsDatensatz noch in das SQuaD Format gebracht werden, sodass wir ein Modell, das auf bereits auf dem SQuaD datensatz trainiert wurde, mit den Amazon Daten weiter finetunen können.


In [ ]:
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 26.8 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-kdwqxqy1/farm-haystack_ab3bbf7a8180432eae4cf7451a41ae43
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-kdwqxqy1/farm-haystack_ab3bbf7a8180432eae4cf7451a41ae43
  Resolved https://github.com/deepset-ai/haystack.git to commit b579b9d54a4e55d6f1d19c16605464b975dadff3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.4/228.4 kB 18

In [ ]:
# Relevante bibliotheken importieren
import pandas as pd
import json
from urllib.request import urlopen
from haystack.utils import SquadData

In [ ]:
#Erstellen einer Methode zum umwandeln der Span Based TrainingsDatensätze (von Gupta et al.) in das SQuaD format
#drei parameter: 
#   url (string), die URL, von dem der Datensatz herutergelden wird
#   ouputfile_name (string), der name der aufbereiteten json-Datei, die ausgegeben wird
#   split (string), angabe ob der datensatz der train split ("train") ist oder,
#     im fall einer der anderen splits (test, val), welche span heuristik verwendet werden soll 
#     ("answers_snippet_spans_bleu2","answers_snippet_spans_bleu4","answers_snippet_spans_rouge","answers_sentence_ir")
def amazonQA_span_based_to_squad(url,outputfile_name,split):
    data = []

    if(split=="train"):
      answer_field='answers'
    else:
      answer_field=split

    #für jedes json objekt in der jsonl die relevanten attribute der liste "data" hinzufüen
    with urlopen(url) as f:
        for line in f:
          json_line = json.loads(line)
          for question in json_line['qas']:
            for answer in question[answer_field]:
                data.append([json_line['context'][0:10], json_line['context'],question['question'], question['id'], answer['text'], answer['answer_start'], question['is_impossible']])
    
    #aus der liste "data" wird ein pandas Dataframe df erstellt, bei dem die spaltennamen dem Squad Format entsprechen (diese Spaltennamen müssen so vorhanden sein, siehe https://github.com/deepset-ai/haystack/issues/2119)
    df=pd.DataFrame(data, columns=["title", "context", "question", "id", "answer_text", "answer_start", "is_impossible"])

    # konvertierung zu einem SquadData Objekt, basierend auf https://github.com/deepset-ai/haystack/issues/2119 (der meiste code wurde hier entnommen)
    # und https://github.com/deepset-ai/haystack/issues/2651 (um den fehler aus der ersten quelle [sq.df_to_squad(df)] zu korrigieren [richtig: sq.df_to_data(df)])
    sq = SquadData({'data': {}})
    squad_docs = sq.df_to_data(df)
    converted_squad = SquadData(squad_docs)
    converted_squad.save(outputfile_name + '.json')


Dieser Datensatz ist viel zu groß, deshalb eine alternative methode, die nur jede zehnte frage hernimmt, um so einen downgesampleten datensatz zu erhalten:

In [ ]:

def downsample_amazonQA_span_based_to_squad(url,outputfile_name,split):
    data = []

    if(split=="train"):
      answer_field='answers'
    else:
      answer_field=split

    #für jedes json objekt in der jsonl die relevanten attribute der liste "data" hinzufüen
    with urlopen(url) as f:
        for i, line in enumerate(f):
          if i % 10 == 0:
            json_line = json.loads(line)
            for question in json_line['qas']:
              for answer in question[answer_field]:
                  data.append([json_line['context'][0:10], json_line['context'],question['question'], question['id'], answer['text'], answer['answer_start'], question['is_impossible']])
    
    #aus der liste "data" wird ein pandas Dataframe df erstellt, bei dem die spaltennamen dem Squad Format entsprechen (diese Spaltennamen müssen so vorhanden sein, siehe https://github.com/deepset-ai/haystack/issues/2119)
    df=pd.DataFrame(data, columns=["title", "context", "question", "id", "answer_text", "answer_start", "is_impossible"])
    sq = SquadData({'data': {}})
    squad_docs = sq.df_to_data(df)
    converted_squad = SquadData(squad_docs)      
    converted_squad.save(outputfile_name + '.json')
    # konvertierung zu einem SquadData Objekt, basierend auf https://github.com/deepset-ai/haystack/issues/2119 (der meiste code wurde hier entnommen)
    # und https://github.com/deepset-ai/haystack/issues/2651 (um den fehler aus der ersten quelle [sq.df_to_squad(df)] zu korrigieren [richtig: sq.df_to_data(df)])

In [ ]:
#URL für den trainingssplitdes Span based Datensatzes von Gupta et al. (siehe https://github.com/amazonqa/amazonqa)
url_train = "https://amazon-qa.s3-us-west-2.amazonaws.com/train-qar_squad.jsonl"

#Aufbereitung des Trainingsdatensatz mit der vorher definierten Methode "amazonQA_span_based_to_squad()":

downsample_amazonQA_span_based_to_squad(url_train,"train_split_squad_format","train")

INFO:haystack.utils.squad_data:Converting data frame to squad format data
INFO:haystack.utils.squad_data:Aggregating the answers of each question
100%|██████████| 45594/45594 [01:02<00:00, 730.94it/s]
INFO:haystack.utils.squad_data:Aggregating the questions of each paragraphs of each document
100%|██████████| 45520/45520 [00:42<00:00, 1070.55it/s]
INFO:haystack.utils.squad_data:Aggregating the paragraphs of each document
100%|██████████| 20062/20062 [00:15<00:00, 1335.83it/s]


In [ ]:
#zippen der datei für schnelleres herunterladen
!zip -r "./train_split_squad_format_downsampled.zip" "./train_split_squad_format.json"

  adding: train_split_squad_format.json (deflated 69%)


In [ ]:
#URL für den validationsplit des Span based Datensatzes von Gupta et al. (siehe https://github.com/amazonqa/amazonqa)
url_val = "https://amazon-qa.s3-us-west-2.amazonaws.com/val-qar_squad.jsonl"

#Aufbereitung des Validationsdatensatz mit der vorher definierten Methode "amazonQA_span_based_to_squad()":

#wir verwenden als span heuristik hier "answers_sentence_ir", da die ergebnisse von Gupta et al. (2019) zeigten, das die IR based score match die effektivste
#span Heuristik für den Amazon QA Datensatz ist
downsample_amazonQA_span_based_to_squad(url_val,"val_split_squad_format","answers_sentence_ir")

INFO:haystack.utils.squad_data:Converting data frame to squad format data
INFO:haystack.utils.squad_data:Aggregating the answers of each question
100%|██████████| 5897/5897 [00:07<00:00, 745.93it/s]
INFO:haystack.utils.squad_data:Aggregating the questions of each paragraphs of each document
100%|██████████| 5893/5893 [00:05<00:00, 1042.16it/s]
INFO:haystack.utils.squad_data:Aggregating the paragraphs of each document
100%|██████████| 3787/3787 [00:02<00:00, 1332.49it/s]


In [ ]:
#zippen der datei für schnelleres herunterladen
!zip -r "./val_split_squad_format_downsampled.zip" "./val_split_squad_format.json"

  adding: val_split_squad_format.json (deflated 70%)


In [ ]:
#URL für den testssplit des Span based Datensatzes von Gupta et al. (siehe https://github.com/amazonqa/amazonqa)
url_test = "https://drive.google.com/u/0/uc?id=1eede6X_r7uoOmDZkv5NlbM4Mu-OP-cCe&export=download&confirm=t"

#Aufbereitung des Testdatensatz mit der vorher definierten Methode "amazonQA_span_based_to_squad()":

#wir verwenden als span heuristik hier "answers_sentence_ir", da die ergebnisse von Gupta et al. (2019) zeigten, das die IR based score match die effektivste
#span Heuristik für den Amazon QA Datensatz ist
downsample_amazonQA_span_based_to_squad(url_test,"test_split_squad_format","answers_sentence_ir")

INFO:haystack.utils.squad_data:Converting data frame to squad format data
INFO:haystack.utils.squad_data:Aggregating the answers of each question
100%|██████████| 5953/5953 [00:07<00:00, 751.01it/s]
INFO:haystack.utils.squad_data:Aggregating the questions of each paragraphs of each document
100%|██████████| 5947/5947 [00:05<00:00, 1113.83it/s]
INFO:haystack.utils.squad_data:Aggregating the paragraphs of each document
100%|██████████| 3818/3818 [00:02<00:00, 1347.31it/s]


In [ ]:
#zippen der datei für schnelleres herunterladen
!zip -r "./test_split_squad_format_downsampled.zip" "./test_split_squad_format.json"

  adding: test_split_squad_format.json (deflated 70%)
